## 実践演習7-4

sklearnでrandomized Searchを行います。

In [1]:
import scipy as sp
import numpy as np
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV

bostonデータを読み込み、パターン行列Xと教師ベクトルyにデータを格納します。

In [2]:
boston = load_boston()
X = boston.data
y = boston.target

### scipy.stats.exponの動作確認

[scipy.stats.expon](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.expon.html)は確率密度関数 $\frac{exp(-x)}{scale}$ for $x \geq 0$を表します。rvsメソッドはその関数に基づいて乱数を発生し、ndarrayに格納します。適当なscaleで乱数を10個発生させてみます。

In [3]:
sp.stats.expon(scale=1).rvs(size=10)

array([0.19112387, 0.51629799, 0.11831997, 0.07894393, 0.59245425,
       0.03775291, 0.04448439, 0.72971174, 1.83040016, 0.24338618])

## 回帰問題への適用

bostonデータに対する[SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)回帰でRandomized searchを行います。

回帰器のインスタンスの作成

In [4]:
svr = SVR()
svr

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

Gridを作成します。スラック変数の重みCと、RBFカーネルの係数gammaでグリッドを作ります。

In [5]:
params = {'C': sp.stats.expon(scale=100), 'gamma': sp.stats.expon(scale=.1)}

回帰の場合は、ShuffleSplitのインスタンスを作成し、それをRandomizedSearchCVのcvパラメータの値として与えます。

In [6]:
cv = ShuffleSplit(n_splits=3)
reg = RandomizedSearchCV(svr, params, cv=cv, scoring='r2', n_iter=10, return_train_score=True)
reg.fit(X,y) 

RandomizedSearchCV(cv=ShuffleSplit(n_splits=3, random_state=None, test_size='default',
       train_size=None),
          error_score='raise',
          estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020A533349E8>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020A53334DD8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='r2', verbose=0)

結果の詳細表示で、交差確認がうまく適用できていることを確認します。

In [7]:
reg.cv_results_

{'mean_fit_time': array([0.01885128, 0.01636179, 0.01833169, 0.01669741, 0.01800974,
        0.02086488, 0.01932446, 0.02248518, 0.01936865, 0.01889157]),
 'mean_score_time': array([0.00132044, 0.00114536, 0.00132465, 0.00134166, 0.00132052,
        0.00132259, 0.00133959, 0.00115856, 0.00098761, 0.00099055]),
 'mean_test_score': array([0.01576812, 0.13782465, 0.29042404, 0.01367641, 0.37779566,
        0.55436326, 0.39178823, 0.55666119, 0.48002716, 0.45656334]),
 'mean_train_score': array([0.99960843, 0.82293602, 0.99907907, 0.99616105, 0.99945251,
        0.9977162 , 0.99877287, 0.99838402, 0.99816095, 0.99722065]),
 'param_C': masked_array(data=[51.222592590202545, 10.234815998080789,
                    143.1990223877667, 24.75700273935731,
                    333.60333366229764, 50.55708939324863,
                    162.30779332457766, 251.00217925273515,
                    59.863600202938414, 25.81595501561772],
              mask=[False, False, False, False, False, False, Fal

結果のまとめ（全結果の表示、最適なパラメータ・スコア）

In [8]:
re = reg.cv_results_
for params, mean_score, std_score in zip(re['params'], re['mean_test_score'], re['std_test_score']):
    print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, std_score, params))

0.016 (+/- 0.022) for {'C': 51.222592590202545, 'gamma': 0.33780790041400544}
0.138 (+/- 0.022) for {'C': 10.234815998080789, 'gamma': 0.09027997912151552}
0.290 (+/- 0.132) for {'C': 143.1990223877667, 'gamma': 0.06462308674805493}
0.014 (+/- 0.019) for {'C': 24.75700273935731, 'gamma': 0.3465614090455009}
0.378 (+/- 0.155) for {'C': 333.60333366229764, 'gamma': 0.043767140542867206}
0.554 (+/- 0.176) for {'C': 50.55708939324863, 'gamma': 0.017049250806919877}
0.392 (+/- 0.158) for {'C': 162.30779332457766, 'gamma': 0.0411255685677249}
0.557 (+/- 0.182) for {'C': 251.00217925273515, 'gamma': 0.015830003433317957}
0.480 (+/- 0.172) for {'C': 59.863600202938414, 'gamma': 0.02705623544538618}
0.457 (+/- 0.165) for {'C': 25.81595501561772, 'gamma': 0.02983109760585849}


In [9]:
reg.best_params_

{'C': 251.00217925273515, 'gamma': 0.015830003433317957}

In [10]:
reg.best_score_

0.5566611918506331